In [1]:
import gurobipy as gp

Test gurobipy

In [2]:
# Create a new model
m = gp.Model()

# Create variables
x = m.addVar(vtype='B', name="x")
y = m.addVar(vtype='B', name="y")
z = m.addVar(vtype='B', name="z")

# Set objective function
m.setObjective(x + y + 2 * z, gp.GRB.MAXIMIZE)

# Add constraints
m.addConstr(x + 2 * y + 3 * z <= 4)
m.addConstr(x + y >= 1)

# Solve it!
m.optimize()

print(f"Optimal objective value: {m.objVal}")
print(f"Solution values: x={x.X}, y={y.X}, z={z.X}")

Set parameter TokenServer to value "dev.cma.mines-paristech.fr"
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
Optimal objective value: 3.0
Solution values: x=1.0, y=0.0, z=1.0


## 1 Modèle de base des centrales thermiques

### 1.1 Planification journalière du parc thermique

In [4]:
class Centrale:
    def __init__(self, name, N, Pmin, Pmax, Cmwh):
        self.name = name
        self.N = N
        self.Pmin = Pmin
        self.Pmax = Pmax
        self.Cmwh = Cmwh

In [7]:
dict_11 = {}
dict_11["A"] = Centrale(
    name="A",
    N=12,
    Pmax=2000,
    Pmin=850,
    Cmwh=1.5
)
dict_11["B"] = Centrale(
    name="B",
    N=10,
    Pmax=1750,
    Pmin=1250,
    Cmwh=1.38
)
dict_11["C"] = Centrale(
    name="C",
    N=5,
    Pmax=4000,
    Pmin=1500,
    Cmwh=2.75
)

In [3]:
model = gp.Model(name="1.1")

In [17]:
from unicodedata import name


dict_N = {}
for X in dict_11:
    dict_N[X] = model.addVar(lb=0,ub=dict_11[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées")
dict_P = {}
for X in dict_11:
    dict_P[X] = model.addVar()


In [19]:

model.setObjective(gp.quicksum([dict_P[X] * dict_11[X].Cmwh for X in dict_11]), gp.GRB.MINIMIZE)

In [24]:
for X in dict_11:
    model.addConstr(dict_P[X]<=dict_N[X] * dict_11[X].Pmax, name=f"borne sup puissance, {X}")
    model.addConstr(dict_P[X]>=dict_N[X] * dict_11[X].Pmin, name=f"borne inf puissance, {X}")
model.addConstr(gp.quicksum([dict_P[X] for X in dict_11])==30000)

<gurobi.Constr *Awaiting Model Update*>

In [26]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 9 columns and 21 nonzeros
Model fingerprint: 0x10e280d1
Variable types: 3 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 3e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [3e+04, 3e+04]
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolved: 6 rows, 5 columns, 14 nonzeros
Variable types: 2 continuous, 3 integer (0 binary)
Found heuristic solution: objective 44160.000000
Found heuristic solution: objective 43950.000000
Found heuristic solution: objective 43740.000000
Found heuristic solution: objective 43530.000000

Root relaxation: objective 4.290000e+04, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

In [29]:
for X in dict_11:
    print(f"Nombre de centrale {X} allumées : {dict_N[X].X}")
    print(f"Puissance total des centrales {X} : {dict_P[X].X}")

Nombre de centrale A allumées : 12.0
Puissance total des centrales A : 12500.0
Nombre de centrale B allumées : 10.0
Puissance total des centrales B : 17500.0
Nombre de centrale C allumées : -0.0
Puissance total des centrales C : 0.0
